# Identifying Close Votes on Amendments

In this tutorial we will learn how to identify amendments that were adopted or rejected with a very close majority using the HowTheyVote.eu data set. Amendments that are either passed or rejected with a close margin are probable to have some amount of controversy about their contents. Identifying those can be a first step in finding interesting debates in Parliament.

To follow allong with the tutorial, you should already be familiar with data analysis in Python using `pandas`. You don’t need prior knowledge about the European Parliament.


## Context: Amendments in the European Parliament

Before a text becomes the official position of the European Parliament, the Parliament's Plenary must vote on it.

Usually, Members of Parliament hold multiple votes on a text, due to the process with which Parliaments position is established. At different points in this process, committees and political groups of the Parliament can propose amendments to texts. Each amendment proposes one or many changes to the original document, either exchanging, removing, or adding content. The Plenary then votes on all amendments, either adopting or rejecting them.

Once the Plenary has decided on all amendments, it votes on the final wording of the text. If the majority approves the text, it becomes the official position of the European Parliament. If not, the text is rejected.

## Finding all Votes on a Specific Text

Throughout this tutorial, we will use Parliaments resolution condemning [The Russian aggression against Ukraine](https://howtheyvote.eu/votes/140111) from March 2022.

As a first step, we need the `votes` table from the HowTheyVote.eu data set:

In [5]:
import pandas as pd

votes_df = pd.read_csv('data/votes.csv')
votes_df.head()

,id,timestamp,display_title,reference,description,is_main,is_featured,procedure_reference,procedure_title,responsible_committee_code,count_for,count_against,count_abstention,count_did_not_vote
0,108425,2019-07-15 17:09:37,Mardi - demande du groupe GUE/NGL,NaN,NaN,False,False,NaN,NaN,NaN,83,142,72,454
1,108530,2019-07-18 11:30:02,The situation in Venezuela,RC-B9-0006/2019,§ 2/1,False,False,2019/2730(RSP),The situation in Venezuela,NaN,506,58,70,117
2,108531,2019-07-18 11:30:16,The situation in Venezuela,RC-B9-0006/2019,§ 2/2,False,False,2019/2730(RSP),The situation in Venezuela,NaN,414,184,51,102
3,108532,2019-07-18 11:30:30,The situation in Venezuela,RC-B9-0006/2019,Résolution,True,True,2019/2730(RSP),The situation in Venezuela,NaN,455,85,105,106
4,108533,2019-07-18 11:24:18,The situation at the USA-Mexican border,RC-B9-0014/2019,§ 13,False,False,2019/2733(RSP),The situation at the USA-Mexican border,NaN,311,269,33,138


The `votes` table contains one row for each vote that takes place in the Plenary. Therefore, there might be multiple rows for a given text. There are multiple fields in the `votes` table that we could use to identify all votes that belong to the same report, but we advise to use the `reference` column. It contains the official document reference for the report used by the Parliament.



In [11]:
report_subset = votes_df[votes_df['reference'] == 'B9-0123/2022'].copy()
report_subset.head()

,id,timestamp,display_title,reference,description,is_main,is_featured,procedure_reference,procedure_title,responsible_committee_code,count_for,count_against,count_abstention,count_did_not_vote
11962,140082,2022-03-01 13:49:54,The Russian aggression against Ukraine,B9-0123/2022,§ 3 - Am 4,False,False,2022/2564(RSP),The Russian aggression against Ukraine,NaN,269,396,12,28
11963,140083,2022-03-01 13:49:54,The Russian aggression against Ukraine,B9-0123/2022,Après le § 5 - Am 5,False,False,2022/2564(RSP),The Russian aggression against Ukraine,NaN,450,171,64,20
11964,140084,2022-03-01 13:49:54,The Russian aggression against Ukraine,B9-0123/2022,Après le § 9 - Am 6,False,False,2022/2564(RSP),The Russian aggression against Ukraine,NaN,264,365,57,19
11965,140085,2022-03-01 13:49:54,The Russian aggression against Ukraine,B9-0123/2022,Après le § 11 - Am 22,False,False,2022/2564(RSP),The Russian aggression against Ukraine,NaN,323,342,19,21
11966,140086,2022-03-01 13:49:54,The Russian aggression against Ukraine,B9-0123/2022,Après le § 11 - Am 23,False,False,2022/2564(RSP),The Russian aggression against Ukraine,NaN,284,374,25,22


In [12]:
len(report_subset)

77

77 plenary votes took place concerned with this specific resolution. From the  `description` field, we can get a (very basic) first glance which part of the report each amendment targeted: For example, "§ 3 - Am 4" means that Amendment 4 contained changes to § 3 of the resolution.

As mentioned in the very beginning, after decising on the amendments, the final text will usually be voted on. This is indicated in our data schame by the `is_main` column. Therefore, the following will retrieve the vote on the final, unified version of the report:

In [13]:
report_subset[report_subset['is_main'] == True]

,id,timestamp,display_title,reference,description,is_main,is_featured,procedure_reference,procedure_title,responsible_committee_code,count_for,count_against,count_abstention,count_did_not_vote
11990,140111,2022-03-01 16:48:54,The Russian aggression against Ukraine,B9-0123/2022,Proposition de résolution (ensemble du texte),True,True,2022/2564(RSP),The Russian aggression against Ukraine,NaN,637,13,26,29


## Identifying Votes With a Close Margin

In the columns `count_for`, `count_against`, `count_abstention`, and `count_did_not_vote` we can see the number of Members of Parliament (MEP) who voted in a specific way on the entirety of this text or who did not partook in the vote. In contrast to not voting at all, MEPs also have the option to actively vote abstain in each vote.

In [14]:
report_subset[report_subset['is_main'] == True][['reference'] + report_subset.filter(regex='^count').columns.tolist()]

,reference,count_for,count_against,count_abstention,count_did_not_vote
11990,B9-0123/2022,637,13,26,29


Note, that this does not give us any information about which specific MEPs voted in which way.

In order to identify the amendments that got accepted or rejected with a very close margin, we are interested in the difference between votes in favour and votes against. Note, that we calculate this on the basis of a simple majority in this tutorial, i.e., as long as more MEPs voted in favor than against an amendment, it is accepted. This is always true for amendments regarding resolutions.

In cases where an amendment was accepted, this difference will be a positive number (as the number of votes in favour will be larger then that of votes against). However, when amendments are narrowly rejected, the opposite will be true, resulting in a negative number for the difference.

As we are interested in narrow decisions in general, we will use the absolute difference between votes in favour and against.

In [15]:
report_subset['abs_diff_for_against'] = (report_subset['count_for'] - report_subset['count_against']).abs()
report_subset[['reference', 'description', 'is_main', 'abs_diff_for_against']].sort_values(by='abs_diff_for_against').head(10)

,reference,description,is_main,abs_diff_for_against
11965,B9-0123/2022,Après le § 11 - Am 22,False,19
11995,B9-0123/2022,§ 13 - Am 29/2,False,22
11989,B9-0123/2022,Après le considérant E - Am 3,False,32
11987,B9-0123/2022,Après le considérant D - Am 1,False,37
11966,B9-0123/2022,Après le § 11 - Am 23,False,90
11964,B9-0123/2022,Après le § 9 - Am 6,False,101
11962,B9-0123/2022,§ 3 - Am 4,False,127
11986,B9-0123/2022,Après le § 45 - Am 28,False,255
11983,B9-0123/2022,Après le § 37 - Am 21,False,260
11975,B9-0123/2022,§ 19 - Am 9,False,270


The closest amendment had an absolute difference of 19 votes against versus votes in favour. Although this seems not extremely close, we can also see a substantive difference between the first four votes, which all have margins below 40, and the following ones, which are close to differences of 100 votes and more.

## Wrapping Up

Identifying these close amendments can be a good starting point to figure out which parts of a text were most controversial in Parliament. Using the `votes` table, those can be identified quite fast, by finding all votes corresponding to a specific text and afterwards finding the narrowest of these decisions.

With this information in hand it is now possible to continue with substantive research on the specific amendments, or to take a look at which MEPs and groups voted which way.